Follow these steps and complete the queries listed below after testing them through SQL Workbench


In [2]:
!pip install pymysql

In [3]:
#import required modules
import subprocess
import pymysql
import math
from getpass import getpass

In [4]:
# Ensure you set these environment properties here.
# We use getpass - so that they do not get saved into github
# fill the userid
dbhost='assignment-1.cburq6ku5g17.us-east-1.rds.amazonaws.com' # paste the db host string - something like this database-1.cjcbghqlwnpl.us-east-1.rds.amazonaws.com here
username=getpass('Enter Username Here:')
passwd=getpass('Enter Password Here:')
databasename='lahman' # change this if you imported the database with a different name

Enter Username Here:··········
Enter Password Here:··········


In [5]:
# Let us define the base database class.
# Make sure that you input the password through the form on the right
# - but do not store the password back inthe github

class DB:
    def __init__(self):
        try:
          self.conn = pymysql.connect(
              user=username,
              password=passwd,
              host=dbhost,
              database = databasename
          )

        except Exception as e:
            print("I am unable to connect to the database - bye " + str(e))
            exit()
        print('Got a DB connection!- hurray')
        self.cur = self.conn.cursor()

    def cursor(self):
        return self.cur

    def getNewCursor(self):
        return self.conn.cursor()

    def connection(self):
        return self.conn

    def query(self, q):
        self.cur.execute(q)
        try:
          return self.cur.fetchall()
        except Exception as e:
          print('no result to return')
          return

The code below will try to connect with the database instance that you've connected in AWS. If, for some reason, this fails to connect (even though you've set up the username, host, password and etc correctly), then that probably means that your Amazon VPC security group is incorrectly configured to either not allow public access at all or only to allow public access from your system (and not colab). To remedy this, you'd have to follow following steps:

1. Navigate to the dashboard of your RDS instance
2. Find Connectivity & Security tab (this is usually active by default)
3. Click the VPC security group under Security section
4. Click "security group ID" link
5. Go to "Inbound Rules"
6. Click "Edit Inbound Rules"
7. Under the Source setting, click "Anywhere IPv4"
8. Click Save and wait for changes to take effect.

In [6]:
# Check that your connection is working correctly using this sample query
db1 = DB() # we instantiate an oject of the DB class. This will call the constructor and it will attempt the connection

# if the connection succeeded - the following should work

q = 'use '+databasename+';' # note that database name was set above
db1.query(q)
q = 'show tables;' # lets see all tables
res=db1.query(q)
print(res) # shows the results.

# Another way to get to all the results is to use a cursor that enables you to iterate over the results
mycursor = db1.cursor()
mycursor.execute('use '+databasename+';')
mycursor.execute("show tables;")
myresult = mycursor.fetchall()
print('Success! I ran a query and got {} results'.format(len(myresult)))
for x in myresult:
  print(x)

Got a DB connection!- hurray
(('allstarfull',), ('appearances',), ('awardsmanagers',), ('awardsplayers',), ('awardssharemanagers',), ('awardsshareplayers',), ('batting',), ('battingpost',), ('fielding',), ('fieldingof',), ('fieldingpost',), ('halloffame',), ('hofold',), ('managers',), ('managershalf',), ('master',), ('pitching',), ('pitchingpost',), ('salaries',), ('schools',), ('schoolsplayers',), ('seriespost',), ('teams',), ('teamsfranchises',), ('teamshalf',))
Success! I ran a query and got 25 results
('allstarfull',)
('appearances',)
('awardsmanagers',)
('awardsplayers',)
('awardssharemanagers',)
('awardsshareplayers',)
('batting',)
('battingpost',)
('fielding',)
('fieldingof',)
('fieldingpost',)
('halloffame',)
('hofold',)
('managers',)
('managershalf',)
('master',)
('pitching',)
('pitchingpost',)
('salaries',)
('schools',)
('schoolsplayers',)
('seriespost',)
('teams',)
('teamsfranchises',)
('teamshalf',)


If everything works as intended. Then go ahead and implement the following queries.
Again - test them first in mysql workbench. Then try it.

1. The number of all stars in allstarfull.
2. The most home runs in a season by a single player (using the batting table).
3. The playerid of the player with the most home runs in a season.
4. The number of leagues in the batting table.
5. Barry Bond's average batting average (playerid = 'bondsba01') where batting average is hits / at-bats. Note you will nead to cast hits to get a decimal: cast(h as real)
6. The teamid with the fewest hits in the year 2000 (ie., yearid = '2000'). Return both the teamid, and the number of hits. Note you can use ORDER BY column and LIMIT 1.
7. The teamid in the year 2000 (i.e., yearid = '2000')  with the highest average batting average. Return the teamid and the average. To prevent divsion by 0, limit at-bats > 0.
8. The number of all stars the giants (teamid = 'SFN') had in 2000.
9. The yearid which the giants had the most all stars.
10. The average salary in year 2000.
11. The number of positions (e.g., catchers, pitchers) that have average salaries greather than 2000000 in yearid 2000. You will need to join fielding with salaries. Also consider using a HAVING clause.
12. The number of errors Barry Bonds had in 2000.
13. The average salary of all stars in 2000. # apply floor on the results. The final result should be an integer
14. The average salary of non-all stars in 2000. # apply floor on the results. The final result should be an integer

The tests for each query is below. You can try that.

In [7]:
#As an example q1 and q3 and q9 have been already filled for you.
#Note the use of group by https://www.w3schools.com/sql/sql_groupby.asp to categorize the data by player id and year.
q1 = 'select count(*) from allstarfull;'
q3 = 'select playerid from batting INNER Join (select MAX(HR) as m from batting) as data ON batting.hr=data.m'
q9 = "SELECT yearid FROM allstarfull WHERE teamid = 'SFN' group by yearid having count(distinct playerid)\
      =(select max(nump) from (SELECT yearid, count(distinct playerid) as nump FROM allstarfull WHERE teamid = 'SFN' group by yearid) as innertable)"

In [13]:
#Fill queries here

q2 = 'SELECT MAX(HR) FROM batting;'

q4 = 'SELECT COUNT(DISTINCT lgID) FROM batting;'

q5 = "SELECT AVG(CAST(H AS REAL) / AB) as avg_ba FROM batting WHERE playerID = 'bondsba01';" #cast as REAL

q6 = 'SELECT teamID, H FROM teams WHERE yearID = 2000 ORDER BY H ASC LIMIT 1;'


q7 = 'SELECT teamID, AVG(CAST(H AS REAL) / AB) as avg_ba FROM batting WHERE yearID = 2000 AND AB > 0 GROUP BY teamID ORDER BY avg_ba DESC LIMIT 1;' #cast as REAL


q8 = "SELECT COUNT(*) FROM allstarfull WHERE teamID = 'SFN' AND yearID = 2000;"


q9 = "SELECT yearid FROM allstarfull WHERE teamid = 'SFN' group by yearid having count(distinct playerid)\
      =(select max(nump) from (SELECT yearid, count(distinct playerid) as nump FROM allstarfull WHERE teamid = 'SFN' group by yearid) as innertable)"


q10 = 'SELECT AVG(salary) from salaries WHERE yearID = 2000'


q11 = 'SELECT COUNT(*) FROM (SELECT f.POS from fielding f JOIN salaries s ON f.yearID = s.yearID AND f.playerID = s.playerID WHERE f.yearID = 2000 GROUP BY f.POS HAVING AVG(s.salary) > 2000000) as temp'


q12 = "SELECT SUM(E) FROM fielding WHERE playerID = 'bondsba01' AND yearID = 2000;"


q13 = 'SELECT FLOOR(AVG(salary)) FROM salaries WHERE yearID = 2000 AND playerID IN (SELECT DISTINCT playerID FROM allstarfull WHERE yearID = 2000);' # apply floor on the results. The final result should be an integer


q14 = 'SELECT FLOOR(AVG(salary)) FROM salaries WHERE yearID = 2000 AND playerID NOT IN (SELECT DISTINCT playerID FROM allstarfull WHERE yearID = 2000);' # apply floor on the results. The final result should be an integer



In [14]:
# just get the printout of all queries and save it
for q in [q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11, q12, q13, q14]:
    print(q)
    if q:   # do this only for non-empty queries i.e. only once you've filled it
      res = db1.query(q)
      print (res)


select count(*) from allstarfull;
((4834,),)
SELECT MAX(HR) FROM batting;
((73,),)
select playerid from batting INNER Join (select MAX(HR) as m from batting) as data ON batting.hr=data.m
(('bondsba01',),)
SELECT COUNT(DISTINCT lgID) FROM batting;
((7,),)
SELECT AVG(CAST(H AS REAL) / AB) as avg_ba FROM batting WHERE playerID = 'bondsba01';
((0.2983057628600463,),)
SELECT teamID, H FROM teams WHERE yearID = 2000 ORDER BY H ASC LIMIT 1;
(('MIL', 1366),)
SELECT teamID, AVG(CAST(H AS REAL) / AB) as avg_ba FROM batting WHERE yearID = 2000 AND AB > 0 GROUP BY teamID ORDER BY avg_ba DESC LIMIT 1;
(('SEA', 0.2807991584890046),)
SELECT COUNT(*) FROM allstarfull WHERE teamID = 'SFN' AND yearID = 2000;
((2,),)
SELECT yearid FROM allstarfull WHERE teamid = 'SFN' group by yearid having count(distinct playerid)      =(select max(nump) from (SELECT yearid, count(distinct playerid) as nump FROM allstarfull WHERE teamid = 'SFN' group by yearid) as innertable)
((1966,),)
SELECT AVG(salary) from salaries 

Run Each of these tests and show the results of your query.

In [15]:
def test_q1():
    db = DB()
    assert db.query(q1)[0][0] == 4834

In [16]:
def test_q2():
    db = DB()
    assert db.query(q2)[0][0] == 73

In [17]:
def test_q3():
    db = DB()
    assert db.query(q3)[0][0] == 'bondsba01'

In [18]:
def test_q4():
    db = DB()
    assert db.query(q4)[0][0] == 7

In [30]:
def test_q5():
    db = DB()
    assert math.isclose(db.query(q5)[0][0], 0.298305762454545) or math.isclose(db.query(q5)[0][0], 0.29830576)

#0.29830576 - this is also correct

In [20]:
def test_q6():
    db = DB()
    assert db.query(q6)[0][0] == 'MIL'
    assert db.query(q6)[0][1] == 1366

In [21]:
def test_q7():
    db = DB()
    assert db.query(q7)[0][0] == 'SEA'
    print(db.query(q7)[0][1])
    assert math.isclose(db.query(q7)[0][1], 0.28079655) or math.isclose(db.query(q7)[0][1], 0.2807991584890046)

#Q7 0.2807991584890046 - is also correct

In [22]:
def test_q8():
    db = DB()
    assert db.query(q8)[0][0] == 2

In [23]:
def test_q9():
    db = DB()
    assert db.query(q9)[0][0] in [1961, 1962, 1966]

In [24]:
def test_q10():
    db = DB()
    assert int(db.query(q10)[0][0]) == 1992984

In [25]:
def test_q11():
    db = DB()
    assert db.query(q11)[0][0] == 3

In [26]:
def test_q12():
    db = DB()
    assert db.query(q12)[0][0] in [3 ,6] # or 6

In [27]:
def test_q13():
    db = DB()
    assert int(db.query(q13)[0][0]) == 5388841
# apply floor on the results

In [28]:
def test_q14():
    db = DB()
    assert int(db.query(q14)[0][0]) == 1692309

In [31]:
# run these
test_q1()
print('Q1 - succesful')
test_q2()
print('Q2 - succesful')
test_q3()
print('Q3 - succesful')
test_q4()
print('Q4 - succesful')
test_q5()
print('Q5 - succesful')
test_q6()
print('Q6 - succesful')
test_q7()
print('Q7 - succesful')
test_q8()
print('Q8 - succesful')
test_q9()
print('Q9 - succesful')
test_q10()
print('Q10 - succesful')
test_q11()
print('Q11 - succesful')
test_q12()
print('Q12 - succesful')
test_q13()
print('Q13 - succesful')
test_q14()
print('Q14 - succesful')

Got a DB connection!- hurray
Q1 - succesful
Got a DB connection!- hurray
Q2 - succesful
Got a DB connection!- hurray
Q3 - succesful
Got a DB connection!- hurray
Q4 - succesful
Got a DB connection!- hurray
Q6 - succesful
Got a DB connection!- hurray
0.2807991584890046
Q7 - succesful
Got a DB connection!- hurray
Q8 - succesful
Got a DB connection!- hurray
Q9 - succesful
Got a DB connection!- hurray
Q10 - succesful
Got a DB connection!- hurray
Q11 - succesful
Got a DB connection!- hurray
Q12 - succesful
Got a DB connection!- hurray
Q13 - succesful
Got a DB connection!- hurray
Q14 - succesful


In [39]:
# Part 2: Use the profiling guide from brightspace and time each of the tests. Write code below to execute each test and log the time required to run the test.

In [38]:
import time

for i, q in enumerate([q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11, q12, q13, q14], 1):
    if q:
        print(f"Q{i}")
        start_time = time.time()
        res = db1.query(q)
        end_time = time.time()

        duration = end_time - start_time

        print(f"Time consumed: {duration:.4f} sec")
        print('-' * 30)

Q1
Time consumed: 0.2555 sec
------------------------------
Q2
Time consumed: 0.2578 sec
------------------------------
Q3
Time consumed: 0.2906 sec
------------------------------
Q4
Time consumed: 0.2652 sec
------------------------------
Q5
Time consumed: 0.2245 sec
------------------------------
Q6
Time consumed: 0.2237 sec
------------------------------
Q7
Time consumed: 0.2679 sec
------------------------------
Q8
Time consumed: 0.2229 sec
------------------------------
Q9
Time consumed: 0.2292 sec
------------------------------
Q10
Time consumed: 0.2219 sec
------------------------------
Q11
Time consumed: 0.2290 sec
------------------------------
Q12
Time consumed: 0.2216 sec
------------------------------
Q13
Time consumed: 0.2253 sec
------------------------------
Q14
Time consumed: 0.2252 sec
------------------------------
